<a href="https://colab.research.google.com/github/alexandrekumagae/imersao-ia-alura-2edicao/blob/main/index_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U -q google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.8/677.8 kB 16.4 MB/s eta 0:00:00


In [3]:
#Importações e configurações iniciais
import numpy as np
import pandas as pd
import google.generativeai as genai

from google.colab import userdata

api_key = userdata.get("SECRET_KEY")
genai.configure(api_key=api_key)

In [4]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [5]:
#Exemplo de embedding
title = "A próxima geração de IA para desenvolvedores e Google Workspace."
sample_text = ("A próxima geração de IA para desenvolvedores e Google Workspace."
      "\n"
      "Artigo completo: \n"
      "\n"
      "Gemini API & Google AI Studio: Uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa")

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")

print(embeddings)

{'embedding': [0.05164398, -0.029184679, -0.021038307, 0.01889091, 0.06670586, -0.010325312, -0.03051333, -0.023181772, 0.06943018, 0.06445222, 0.012257243, 0.016009925, -0.04652012, -0.024178423, 0.009291551, -0.030349659, 0.01712187, -0.016522488, -0.028315177, -0.0016221493, 0.006789069, 0.0026177294, -0.029617885, -0.06398246, -0.020842616, 0.02251817, 0.014000838, -0.039801057, -0.031754486, 0.026591344, -0.049806118, 0.05577756, -0.03724814, 0.009917345, -0.03552398, -0.0398147, -0.01906688, -0.04791909, -0.0039505763, -0.005695755, 0.00887427, -0.08593744, -0.005529157, 0.027297348, -0.005218387, -0.019078588, 0.04664637, 0.04115992, 0.014044701, -0.059500065, 0.029010002, 0.024212375, 0.06151693, -0.04308137, -0.0050589573, -0.0070583825, 0.023303514, -0.03452451, 0.031800754, 0.000699632, -0.0011116504, 0.023142401, -0.020819003, 0.04761921, 0.020421114, -0.059803978, -0.03843404, -0.0042681987, 0.0052153366, 0.05228813, 0.011809383, 0.01580301, 0.068652466, -0.06752682, -0.04

In [1]:
#Listagem de documentos que serão buscados
DOCUMENT1 = {
    "Título": "Operação do sistema de controle climático.",
    "Conteúdo": "O Googlecar tem um sistema de controle climático que permite ajustar a temperatura e o fluxo de ar no carro."
}

DOCUMENT2 = {
    "Título": "Touchscreen",
    "Conteúdo": "O seu Googlecar tem uma grande tela sensível ao toque que fornece acesso a uma variedade de recursos, incluindo navegação."
}

DOCUMENT3 = {
    "Título": "Mudança de marchas",
    "Conteúdo": "Seu Googlecar tem uma transmissão automática. Para trocar as marchas, basta mover a alavanca de câmbio para a posição desejada."
}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [8]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,Operação do sistema de controle climático.,O Googlecar tem um sistema de controle climáti...
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...


In [9]:
model = "models/embedding-001"

In [12]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [14]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis = 1)
df

,Titulo,Conteudo,Embeddings
0,Operação do sistema de controle climático.,O Googlecar tem um sistema de controle climáti...,"[-0.008730183, -0.013610988, -0.06007225, 0.05..."
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...,"[0.011367341, -0.048442047, -0.023845239, -0.0..."
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...,"[0.00018316552, -0.03903683, 0.0017067897, 0.0..."


In [16]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta["embedding"])

  indice = np.argmax(produtos_escalares)

  return df.iloc[indice]["Conteudo"]

In [22]:
consulta = "Como se troca marchas em um carro do google?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Seu Googlecar tem uma transmissão automática. Para trocar as marchas, basta mover a alavanca de câmbio para a posição desejada.


In [26]:
generation_config = {
    "temperature": 0.5,
    "candidate_count": 1
}

In [27]:
prompt = f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

Seu Googlecar é como um carro normal, mas sem volante! E adivinha só? Ele até tem marchas, mas sem pedal de embreagem. É só dar um toque na alavanca e pronto, você tá no comando da velocidade!
